# **Configurando as bibliotecas do projeto**

## *Instalação de bibliotecas*

In [116]:
!pip install TA-Lib

Defaulting to user installation because normal site-packages is not writeable


## *Importação de bibliotecas*

In [117]:
'''
    Essa célula será usada para a importação de bibliotecas.
'''

# Instala a biblioteca que será utilizada para se obter os dados financeiros.
import yfinance as yf
# Importa a biblioteca que será utilizada para a manipulação de DataFrames.
import pandas as pd
# Importa a biblioteca que será utilizada para lidar com objetos do tipo "datetime".
from datetime import datetime, timedelta
# Importa a biblioteca que será responsável pela tipagem de funções.
from typing import Union, Optional
# Importa a biblioteca que será utilizada para a manipulação de arrays e também para o uso de algumas funções matemáticas.
import numpy as np
# Importa a biblioteca que será utilizada para calcular métricas de análise técnica.
import talib

# **Data Engineering**

### *Configurações iniciais*

In [118]:
'''
    Essa célula será usada para se obter os tickets das companhias que fazem parte do S&P 500, com base em dados da wikipedia.
'''

# Salva em uma variável a url que contém a tabela com os tickets das companhias.
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Lê todas as tabelas presentes na url acima.
sp500_table = pd.read_html(url)

# Salva a coluna "Symbol" da primeira tabela em uma variável (tal coluna contém todos os tickets das companhias que fazem parte do S&P 500).
sp500_tickets = sp500_table[0]['Symbol']

# Transforma os tickets obtidos em uma lista.
sp500_tickets = sp500_tickets.tolist()

# Exibe a lista de tickets criada acima.
sp500_tickets

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BK',
 'BBWI',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BA',
 'BKNG',
 'BWA',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG'

In [119]:
'''
    Essa célula será usada para definir o intervalo de tempo dos dados que serão coletados, especificando a data de início e a data de fim.
'''

# Define a data inicial cujos dados serão coletados.
start_date = datetime(2019,1,2).date()
# Define a data final cujos dados serão coletados.
end_date = datetime(2023,12,30).date()

In [120]:
'''
    Essa célula será usada para criar um dicionário chamado setup, que contém os parâmetros principais para a coleta de dados, 
    incluindo a lista de tickers das companhias do S&P 500 e o intervalo de tempo.
'''

# Cria um dicionário para guardar alguns parâmetros importantes para a coleta de dados.
setup = {
    "tickets": sp500_tickets,
    "start_date": start_date,
    "end_date": end_date
}

### *Obtendo dados dos tickers via yfinance (Open, High, Low, Close, Adj Close, Volume)*

In [121]:
'''
    Essa célula será usada para a obtenção de alguns dados (Open, High, Low, Close, Adj Close, Volume) 
    dos tickers presentes na variável de configuração setup.
'''

# Cria um array que guardará os DataFrames que contém os dados referentes a cada um dos tickers presentes na variável de configuração setup.
data_array = np.empty(len(sp500_tickets), dtype=object)

# Itera sobre cada um dos tickers presentes na variável de configuração setup.
for i, ticker in enumerate(sp500_tickets):
    # Faz o download dos dados do ticker em questão
    ticker_data = yf.download(ticker, setup['start_date'], setup['end_date'])
    # Adiciona um nome ao DataFrame do ticker em questão.
    ticker_data.name=ticker
    # Salva o DataFrame do ticker em questão na variável "data_array".
    data_array[i] = ticker_data

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

$BF.B: possibly delisted; No price data found  (1d 2019-01-02 -> 2023-12-30)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

### *Filtrando os dados obtidos*

In [122]:
'''
    Essa célula será responsável por filtrar a variável "data_array" de modo que esta só contenha DataFrames não vazios.
'''

# Cria um filtro para identificar quais DataFrames presentes em "data_array" são vazios, isto é, não possuem nenhum índice.
void_filter = np.array([df.index.empty for df in data_array])

# Aplica a negação do filtro gerado acima (ou seja, filtra os DataFrames que não são vazios) no array "data_array" e armazena o resultado 
# na variável "filtered_data_array". 
filtered_data_array = data_array[~void_filter]

# Exibe uma mensagem que informa quantos tickers foram removidos na filtragem realizada acima.
print(f"Foram removidos {len(data_array) - len(filtered_data_array)} tickers que estavam vazios.")

Foram removidos 5 tickers que estavam vazios.


In [123]:
'''
    Essa célula será responsável por filtrar a variável "data_array" de modo que esta só contenha DataFrames cujos índices se iniciem em 
    setup['start_date'].
'''

# Armazena o tamanho atual do array "filtered_data_array" para que se tenha ideia ao final dessa célula, de quantos tickers 
# foram removidos com o filtro que será aplicado aqui.
filtered_data_array_initial_len = len(filtered_data_array)

# Cria um filtro para saber quais dos DataFrames presentes em "filtered_data_array" possuem datas iniciais (isto é, índices de posição 0) 
# iguais à "setup['start_date']".
start_date_filter = [df.index[0].date() == setup['start_date'] for df in filtered_data_array]

# Aplica o filtro criado acima, removendo assim todos os DataFrames presentes em "filtered_data_array" cujos índices iniciais são diferentes de 
# "setup['start_date']".
filtered_data_array = filtered_data_array[start_date_filter]

# Exibe uma mensagem informando quantos DataFrames não satisfaziam a condição do filtro criado acima, e informando também quantos DataFrames
# ainda existem em "data_array".
print(f"Foram removidos {filtered_data_array_initial_len - len(filtered_data_array)} tickers que não possuiam data inicial igual à {setup['start_date']}. Agora, todos os demais {len(filtered_data_array)} tickers \npossuem data de inicio em {setup['start_date']}.")

Foram removidos 13 tickers que não possuiam data inicial igual à 2019-01-02. Agora, todos os demais 485 tickers 
possuem data de inicio em 2019-01-02.


In [124]:
'''
    Essa célula será utilizada para mostrar que, nos dados baixados, não existem DataFrames em "data_array" cujos índices terminam em uma 
    data anterior à última data de negociação possível antes de setup['end_date'], isto é não existem DataFrames em "data_array" cujos índices
    terminam em uma data anterior à 2023-12-29.
'''

# Obtem qual é o menor dentre os últimos índices de cada um dos DataFrames presentes em "filtered_data_array"
lowest_last_date = np.min(np.array([df.index[len(df.index)-1] for df in filtered_data_array]))

# Exibe uma mensagem que informa qual é a menor data final entre todos os DataFrames presentes em "filtered_data_array" e gera uma
# conclusão a partir de tal data.
print(f"Dentre os tickers que passaram nos filtros acima, a menor data final é {lowest_last_date.date()}, que é a última data possível de negociação dentro do \nperiodo estabelecido na variável 'setup' ({setup['end_date']}). Ou seja, nenhum filtro precisa ser aplicado em relação a data final dos tickers.")

Dentre os tickers que passaram nos filtros acima, a menor data final é 2023-12-29, que é a última data possível de negociação dentro do 
periodo estabelecido na variável 'setup' (2023-12-30). Ou seja, nenhum filtro precisa ser aplicado em relação a data final dos tickers.


In [125]:
'''
    Essa célula verifica se algum dos DataFrames presentes em "filtered_data_array" possui valores nulos.
'''

# Cria uma lista auxiliar que conterá valores booleanos em todas as suas posições. Sendo que, para qualquer posição "i" dessa lista, 
# se tal posição contiver o valor True, indica que o DataFrame que ocupa essa mesma posição "i" na variável "filtered_data_array" 
# possui valores nulos. Caso contrário, indica que o DataFrame que ocupa tal posição em "filtered_data_array" não possui valores nulos.
aux_list = [df.isnull().values.any() for df in filtered_data_array]

# Exibe mensagens indicando se existem ou não DataFrames com valores nulos na variável "filtered_data_array".
if True in aux_list: 
    print("Existem DataFrames em 'filtered_data_array' que possuem valores nulos.")
else: 
    print("Não existem DataFrames em 'filtered_data_array' que possuam valores nulos.")

Não existem DataFrames em 'filtered_data_array' que possuam valores nulos.


In [126]:
'''
    Essa célula verifica se todos os DataFrames presentes em "filtered_data_array" possuem os mesmos índices.
'''

# Vamos tomar como referência os índices do DataFrame que ocupa a posição 0 da variável "filtered_data_array".
reference_indexes = filtered_data_array[0].index

# Verifica se todos os DataFrames presentes em "filtered_data_array" possuem os mesmos índices.
areAllIndexesEquals = all(df.index.equals(reference_indexes) for df in filtered_data_array)

if areAllIndexesEquals:
    print("Todos os DataFrames presentes em 'filtered_data_array' possuem os mesmos índices.")
else:
    print("Existem DataFrames presentes em 'filtered_data_array' que possuem índices distintos.")

Todos os DataFrames presentes em 'filtered_data_array' possuem os mesmos índices.


# **Feature Engineering**

#### *Gerando os retornos aritméticos diários dos tickers*

In [127]:
def get_ticker_daily_arithmetic_returns(daily_adjusted_closing_prices: pd.Series) -> pd.Series:
    '''
        Description:
            Essa função calcula e retorna a série temporal dos retornos aritméticos diários de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
        Return:
            ticker_daily_arithmetic_returns (pd.Series): Série temporal dos retornos aritméticos diários do ticker em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''

    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")

    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Cria uma série temporal contendo os retornos diários do ticker em questão.
    ticker_daily_arithmetic_returns = daily_adjusted_closing_prices.pct_change()

    # Como não é possível calcular o retorno diário do "dia 0", removeremos a primeira linha da série temporal gerada acima.
    ticker_daily_arithmetic_returns = ticker_daily_arithmetic_returns[1:]
    
    # Retorna a série temporal resultante.
    return ticker_daily_arithmetic_returns

#### *Gerando os retornos logarítmicos diários dos tickers*

In [128]:
def get_ticker_daily_logarithmic_returns(daily_adjusted_closing_prices: pd.Series) -> pd.Series:
    '''
        Description:
            Essa função calcula e retorna a série temporal dos retornos logarítmicos diários de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
        Return:
            ticker_daily_arithmetic_returns (pd.Series): Série temporal dos retornos logarítmicos diários do ticker em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''

    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")
    
    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Obtem a série temporal dos retornos diários do ticker em questão
    ticker_daily_arithmetic_returns = get_ticker_daily_arithmetic_returns(daily_adjusted_closing_prices)
    
    # Transforma a série temporal dos retornos diários do ticker em questão em sua versão logarítmica.
    ticker_daily_logarithmic_returns = ticker_daily_arithmetic_returns.apply(lambda x: np.log(1 + x))
    
    # Retorna a série temporal resultante.
    return ticker_daily_logarithmic_returns

#### *Gerando uma métrica de volatilidade móvel*

In [129]:
def get_ticker_moving_volatility(daily_adjusted_closing_prices: pd.Series, time_interval: int) -> pd.Series:
    '''
        Description:
            Essa função calcula a volatilidade móvel de "time_interval" dias de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
            time_interval (int): Quantidade de dias da janela móvel.
        Return:
            moving_volatility (pd.Series): Série temporal que representa a evolução da volatilidade móvel de "time_interval" dias do ticker
                                           em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            TypeError: É esperado que o parâmetro "time_interval" seja do tipo int.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''
    
    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")

    # Caso o parâmetro 'time_interval' não seja do tipo int, um erro é retornado.
    if not isinstance(time_interval, int):
        raise TypeError("É esperado que o parâmetro 'time_interval' seja do tipo int.")
    
    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Calcula a volatilidade móvel de "time_interval" dias da série temporal "daily_adjusted_closing_prices".
    moving_volatility = daily_adjusted_closing_prices.rolling(window=time_interval).std()
    
    # Remove os primeiros (time_interval - 1) dados da série temporal gerada acima, isto é, remove os dados NaN de tal série temporal.
    moving_volatility = moving_volatility[time_interval-1:]

    # Retorna o resultado obtido.
    return moving_volatility

#### *Gerando uma métrica de volume de negociação móvel*

In [130]:
def get_ticker_moving_volume(daily_volumes: pd.Series, time_interval: int) -> pd.Series:
    '''
        Description:
            Calcula o volume de negociação móvel de "time_interval" dias para uma série temporal de preços de um certo ticker.  
        Args:
            daily_volumes (pd.Series): Série temporal com os volumes de negociação diários do ticker em questão.
            time_interval (int): Número de dias para a janela móvel.
        Return:
            moving_volume (pd.Series): Série temporal do volume de negociação móvel calculado para o intervalo de dias especificado. 
        Errors:
            TypeError: É esperado que o parâmetro "daily_volumes" seja um objeto do tipo pd.Series.
            TypeError: É esperado que o parâmetro "time_interval" seja do tipo int.
            ValueError: É esperado que a série temporal "daily_volumes" contenha apenas números.
    '''
    
    # Verifica se o parâmetro 'daily_volumes' é do tipo pd.Series.
    if not isinstance(daily_volumes, pd.Series):
        raise TypeError("O parâmetro 'daily_volumes' deve ser do tipo pd.Series.")
    
    # Verifica se o parâmetro 'time_interval' é do tipo int.
    if not isinstance(time_interval, int):
        raise TypeError("O parâmetro 'time_interval' deve ser do tipo int.")
    
    # Verifica se a série temporal contém apenas dados numéricos.
    if not pd.api.types.is_numeric_dtype(daily_volumes):
        raise ValueError("A série temporal 'daily_volumes' deve conter apenas números.")
    
    # Calcula o volume de negociação móvel usando a janela de 'time_interval' dias.
    moving_volume = daily_volumes.rolling(window=time_interval).mean()
    
    # Remove os valores NaN resultantes do cálculo acima.
    moving_volume = moving_volume.dropna()

    return moving_volume


#### *Gerando algumas métricas de análise técnica (EMA , RSI , ATR , MOM) e adicionando novas features aos dados*

In [131]:
# Cria uma cópia do array "filtered_data_array" para inserir novas features sem eventual prejuizo as já existentes.
dfs = filtered_data_array

In [132]:
for df in dfs:
    # Calcula os retornos logarítmicos diários com base nos preços ajustados de fechamento e adiciona uma nova coluna 'Log Returns' ao DataFrame.
    df['Log Returns'] = get_ticker_daily_logarithmic_returns(df['Adj Close'])
    
    # Calcula a volatilidade móvel de 14 dias com base nos preços ajustados de fechamento e adiciona uma nova coluna 'MOV_VOL_14' ao DataFrame.
    df['MOV_VOLATILITY_14'] = get_ticker_moving_volatility(df['Adj Close'], 14)
    
    # Calcula o volume de negociação móvel de 14 dias com base nos preços ajustados de fechamento e adiciona uma nova coluna 'MOV_VOLUME_14' 
    # ao DataFrame.
    df['MOV_VOLUME_14'] = get_ticker_moving_volume(df['Volume'], 14)
    
    # Calcula a Média Móvel Exponencial (EMA) de 14 dias. A EMA é usada no lugar da Média Móvel Simples (SMA) para reduzir o atraso de sinal 
    # típico das médias móveis.
    # [Indicador de média móvel]
    df['EMA_14'] = talib.EMA(df['Adj Close'], timeperiod=14)
    
    # Calcula o Índice de Força Relativa (RSI) de 14 dias, que é um indicador de momentum utilizado para medir a velocidade e a mudança de
    # movimentos de preços.
    # [Indicador de momentum]
    df['RSI_14'] = talib.RSI(df['Adj Close'], timeperiod=14)
    
    # Calcula o Average True Range (ATR) de 14 dias, que é um indicador de volatilidade utilizado para medir a volatilidade do ativo com 
    # base na amplitude de preços (máxima, mínima e fechamento).
    # [Indicador de volatilidade]
    df['ATR_14'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    
    # Calcula o Momentum (MOM) de 14 dias, que é um indicador de momentum usado para identificar a força da tendência de um ativo.
    # [Indicador de momentum]
    df['MOM_14'] = talib.MOM(df['Adj Close'], timeperiod=14)
    
    # Adiciona uma coluna 'Ticker' ao DataFrame para identificar o ticker associado a cada conjunto de dados.
    df['Ticker'] = [df.name for _ in range(len(df))]
    
# Exibe o resultado obtido em um dos DataFrames presentes em "dfs"
dfs[0]

# Observação: valores NaN foram inseridos nos DataFrames juntamente com as novas features. Esses valores NaN são inerentes ao cálculo de alguns
# indicadores técnicos, uma vez que eles requerem um período de ajuste.

,Open,High,Low,Close,Adj Close,Volume,Log Returns,MOV_VOLATILITY_14,MOV_VOLUME_14,EMA_14,RSI_14,ATR_14,MOM_14,Ticker
Date,,,,,,,,,,,,,,
2019-01-02,157.040131,159.690628,156.103683,159.657196,126.860481,2960339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MMM
2019-01-03,157.424744,157.424744,152.918060,153.645493,122.083656,4016407,-0.038381,NaN,NaN,NaN,NaN,NaN,NaN,MMM
2019-01-04,156.145493,160.518402,155.543472,159.966553,127.106262,3582140,0.040317,NaN,NaN,NaN,NaN,NaN,NaN,MMM
2019-01-07,160.000000,160.785950,157.742477,159.598663,126.813889,2585991,-0.002303,NaN,NaN,NaN,NaN,NaN,NaN,MMM
2019-01-08,161.371231,162.299332,158.511703,160.267563,127.345444,2965841,0.004183,NaN,NaN,NaN,NaN,NaN,NaN,MMM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,88.528427,89.807693,88.419731,88.904678,86.383255,2728913,0.007268,1.459182,4.854880e+06,84.524237,66.656455,1.665403,2.347847,MMM
2023-12-26,88.879601,90.794312,88.854515,90.392975,87.829346,3332176,0.016602,1.500869,4.830336e+06,84.964918,70.211749,1.685002,5.215645,MMM
2023-12-27,90.301003,91.220734,90.033447,90.919731,88.341164,2922785,0.005810,1.619461,4.733033e+06,85.415084,71.375104,1.649451,4.833824,MMM


#### *Eliminando redundâncias nas features e lidando com os valores nulos*

*Como o indicador ATR_14 é calculado a partir das variáveis "High" e "Low", essas duas variáveis se tornam redundantes para o treinamento do modelo. Além disso, utilizaremos a coluna "Adj Close" como variável alvo. Como "Adj Close" é uma versão ajustada da variável "Close", esta última também se torna redundante e pouco relevante como feature. Da mesma forma, sem a variável "Close", a variável "Open" perde parte de sua relevância. Ademais, temos duas features que medem aspectos semelhantes de maneiras diferentes: "Volume" e "MOV_VOLUME_14". Como a feature "MOV_VOLUME_14" parece ser mais relevante para o escopo deste projeto, a manutenção da feature "Volume" não é necessária. Portanto, remover essas features do conjunto de dados é uma boa abordagem.*

In [133]:
for df in dfs:
    # Remove as colunas 'Open', 'High', 'Low' e 'Close' de cada DataFrame presente em "dfs".
    df.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1, inplace=True)

# Exibe o resultado obtido em um dos DataFrames presentes em "dfs"
dfs[0]

,Adj Close,Log Returns,MOV_VOLATILITY_14,MOV_VOLUME_14,EMA_14,RSI_14,ATR_14,MOM_14,Ticker
Date,,,,,,,,,
2019-01-02,126.860481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MMM
2019-01-03,122.083656,-0.038381,NaN,NaN,NaN,NaN,NaN,NaN,MMM
2019-01-04,127.106262,0.040317,NaN,NaN,NaN,NaN,NaN,NaN,MMM
2019-01-07,126.813889,-0.002303,NaN,NaN,NaN,NaN,NaN,NaN,MMM
2019-01-08,127.345444,0.004183,NaN,NaN,NaN,NaN,NaN,NaN,MMM
...,...,...,...,...,...,...,...,...,...
2023-12-22,86.383255,0.007268,1.459182,4.854880e+06,84.524237,66.656455,1.665403,2.347847,MMM
2023-12-26,87.829346,0.016602,1.500869,4.830336e+06,84.964918,70.211749,1.685002,5.215645,MMM
2023-12-27,88.341164,0.005810,1.619461,4.733033e+06,85.415084,71.375104,1.649451,4.833824,MMM


*Observe que há dados faltantes nos DataFrames, principalmente devido ao período de ajuste dos indicadores técnicos. Como esses indicadores são calculados com uma janela de 14 dias, os primeiros 14 dias podem conter valores nulos. Para resolver isso, podemos remover as linhas com valores faltantes. Como estamos lidando com no máximo 14 linhas faltantes em um total de 1.258 linhas por DataFrame, essa remoção representa apenas uma pequena fração da amostra. Portanto, a quantidade de dados disponível para o treinamento do modelo não será significativamente afetada.*

In [134]:
for df in dfs:
    # Remove todas as linhas com valores NaN de cada DataFrame na lista "dfs".
    df.dropna(inplace=True)

# Exibe o resultado obtido em um dos DataFrames presentes em "dfs"
dfs[0]

,Adj Close,Log Returns,MOV_VOLATILITY_14,MOV_VOLUME_14,EMA_14,RSI_14,ATR_14,MOM_14,Ticker
Date,,,,,,,,,
2019-01-23,127.750664,0.000156,1.812780,2.878080e+06,127.136232,52.022787,3.724319,0.890182,MMM
2019-01-24,127.903458,0.001195,1.099692,2.726410e+06,127.238529,52.378903,3.557437,5.819801,MMM
2019-01-25,130.149033,0.017404,1.294797,2.668746e+06,127.626596,57.385264,3.580450,3.042770,MMM
2019-01-28,128.355255,-0.013878,1.275533,2.753696e+06,127.723751,52.625909,3.613165,1.541367,MMM
2019-01-29,130.846649,0.019224,1.493873,2.910142e+06,128.140137,57.854209,3.754628,3.501205,MMM
...,...,...,...,...,...,...,...,...,...
2023-12-22,86.383255,0.007268,1.459182,4.854880e+06,84.524237,66.656455,1.665403,2.347847,MMM
2023-12-26,87.829346,0.016602,1.500869,4.830336e+06,84.964918,70.211749,1.685002,5.215645,MMM
2023-12-27,88.341164,0.005810,1.619461,4.733033e+06,85.415084,71.375104,1.649451,4.833824,MMM


####